In [ ]:
!pip install tensorflow==2.12.0
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("photo.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 962ms/step
Class: bulldog
Confidence Score: 0.9834419


In [8]:
import requests
import json

# 你的 LINE Channel Access Token
CHANNEL_ACCESS_TOKEN = 'uHIShBwnw+HmAb5LhtQdPE2CscpJ7h7N+B0rAaLAqoDNMzR8jUdb55XvJB5hE6yQWXAzqeJvQ53DkkAwMhZMjT2WpfXh9KJN2ZpaTk8IWrDGYgsj1sVKiSaqscoBqTPB6jjg1eqdaVjZfhxuXiKrsgdB04t89/1O/w1cDnyilFU='

# LINE Messaging API 的 URL
LINE_API_URL = 'https://api.line.me/v2/bot/message/push'

# 發送訊息的函數
def send_line_message(to, message):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {CHANNEL_ACCESS_TOKEN}'
    }

    data = {
        "to": to,
        "messages": [
            {
                "type": "text",
                "text": message
            }
        ]
    }

    response = requests.post(LINE_API_URL, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        print("訊息已成功發送！")
    else:
        print(f"發送失敗，狀態碼：{response.status_code}")
        print(response.text)

# 使用範例
if __name__ == "__main__":
    # 要發送的用戶 ID
    user_id = 'Uedc065b5c3caec076356539c1f41b8c2'
    # 要發送的訊息
    message = "Class:" + class_name[2:] + "\nConfidence Score:" + str(confidence_score)

    send_line_message(user_id, message)

訊息已成功發送！
